In [4]:
import openai 
import os 
openai.api_key = 'your key'  

class ChatGPT():
    def __init__ (self, model_engine='text-davinci-003'):
        self.model_engine = model_engine
        self.msg_history_turbo = []
        self.msg_history_davinci = ""
        self.role = 'user'
        
    def get_answer(self, prompt=None):
        if self.model_engine == 'gpt-3.5-turbo':
            self.msg_history_turbo.append({"role": self.role, "content":prompt})
            response   = openai.ChatCompletion.create(
            model      = self.model_engine,
            messages   = self.msg_history_turbo)
            reply_text = response['choices'][0]['message']['content']
            self.msg_history_turbo.append({"role":"assistant", "content":reply_text})

        elif self.model_engine == 'text-davinci-003': 
            self.msg_history_davinci += "You: "+prompt 
            response   = openai.Completion.create(
            engine     = self.model_engine,
            prompt     = self.msg_history_davinci,
            max_tokens = 512,
            temperature= 1.0,
            n          = 1,
            #stop       = ["You:", "AI:"]
            )
            reply_text = response.choices[0].text.strip().replace('Answer: ', '')
            self.msg_history_davinci+="\n"+"AI: "+reply_text+"\n"
        # print("ChatGPT: {}".format(reply_text))
        print(len(self.msg_history_turbo))
        return reply_text

print("Done.")

Done.


## Language to Rewards 
This method translates natural langauge instrucitons to rewards to provide parameters for reward functions.



## Reward Translator 
The reward translator is compromised of two part, motion descriptors and reward corders 

### Motion descriptor 
1. Place an object stably on the workspace.
2. We provide also the description of the objects in a given scenario.


### Example 
Describe the motion of a dog robot using the following form:
* The torso of the robot should pitch upward at [NUM: 0.0] degrees.
* The height of the robot's CoM or torso center should be at [NUM: 0.3] m. ...
Remember:
1. If you see phrases like [NUM: default_value], replace the entire phrase with a numerical value.
2. If you see phrases like {CHOICE: choice1, choice2, ...}, it means you should replace the entire phrase with one of the choices listed.

### Reward Corder 
We have a description of a robot's motion and we want you to turn that into the corresponding program with following functions: 

set_torso_rewards(height, pitch)
height: height target for the robot torso
pitch: pitch angle of the torso ...


Example answer code:
import numpy as np set_torso_targets(0.1, np.deg2rad(5)) 

...

Remember:
1. Always format the code in code blocks

In [6]:
engine = "gpt-3.5-turbo"
llm = ChatGPT(model_engine=engine)

prompt = '''
We have a manipulator and we want you to help plan how it should move to perform tasks using the following template:
The rack has slots aligned along the y-axis

The plate size is {x:0.14, y:0.02, z:0.14}m.
The dish rack size is {x:0.12, y:0.225, z:0.085}m.
The rack has slots aligned along the y-axis.

Place a dish plate into a dish rack, and tell me the range.

[start of plan]
[optional] To perform this task, the manipulator's gripper should move close to dish plate
[optional] dish plate should be close to dish rack.
[optional] dish rack is placed on the position {x: 0.8, y: 0.0, z:0.82}m in the table.
[optional] The range of the dish plate is able to be placed in the dish rack {MINIMUM RANGE: 0.0, 0.0, 0.0}m and {MAXIMUM RANGE: 0.0, 0.0, 0.0}.
[end of plan]

Rules:
1. If you see phrases like {MINIMUM RANGE: default x value, default y value, default z value}, {MAXIMUM RANGE: default x value, default y value, default z value}
   replace the entire phrase with a numerical value.
2. If you see [optional], it means you only add that line if necessary for the task, otherwise remove that line.
3. Do not invent new objects not listed here.
4. I will tell you a behavior/skill/task that I want the manipulator to perform and you will provide the full plan,
   even if you may only need to change a few lines. Always start the description with [start of plan] and end it with [end of plan].
5. You can assume that the robot is capable of doing anything, even for the most challenging task.
6. Your plan should be as close to the provided template as possible. Do not include additional details.
7. The robot is facing towards positive x-axis, positive y-axis is to the left of the robot, positive z-axis is upward.
8. Objects are free to move around unless a command is given.
9. Answer the range {MINIMUM RANGE: 0.0, 0.0, 0.0}m and {MAXIMUM RANGE: 0.0, 0.0, 0.0} where a dish is able to be placed in the dish rack.
If you understand, say Yes.
'''

response = llm.get_answer(prompt)
print("Answer: ", response)

2
Answer:  Yes.


In [7]:

# llm = ChatGPT(model_engine=engine)
prompt = '''
We have a plan of a robot arm with gripper to place objects and we want you to turn that into the corresponding reward specifying program with following functions:

```
def set_l2_distance_reward(name_obj_A, name_obj_B)
```
where name_obj_A and name_obj_B are selected from ['dish rack', 'dish plate'].
This term sets a reward for minimizing l2_distance between name_obj_A and name_obj_B so they get closer to each other.

```
def set_obj_orientation_reward(name_obj, x_axis_rotation_radians, z_axis_rotation_radians)
```
this term encourages the orientation of name_obj to be close to the target.

```
def execute_plan(duration=2)
```
This function sends the parameters to the robot and execute the plan for `duration` seconds, default to be 2

```
def set_obj_position_reward(name_obj, x_offset, y_offset, z_height)
```
this term encourages the position of name_obj to be close to the specified target position.

```
def reset_reward()
```
This function resets the reward to default values.

This is the first plan for a new task.

Example answer code:
```
import numpy as np

reset_reward() # This is a new task so reset reward; otherwise we don't need it
set_l2_distance_reward("dish rack", "dish plate")
set_obj_orientation_reward("dish plate", np.deg2rad(30), 0)
set_obj_position_reward("dish plate", 0, 0, 1.0)

execute_plan(4)
```

Remember:
1. Always format the code in code blocks. In your response execute_plan should be called exactly once at the end.
2. Do not invent new functions or classes. The only allowed functions you can call are the ones listed above. Do not leave unimplemented code blocks in your response.
3. The only allowed library is numpy. Do not import or use any other library.
4. If you are not sure what value to use, just use your best judge. Do not use None for anything.
5. Do not calculate the position or direction of any object (except for the ones provided above). Just use a number directly based on your best guess.
6. You do not need to make the robot do extra things not mentioned in the plan such as stopping the robot.

If you understand, say Yes.
'''
response = llm.get_answer(prompt)
print("Answer: ", response)

4
Answer:  Yes.


In [8]:
prompt = "Put a dish plate into a dish rack"
response = llm.get_answer(prompt)
print("Answer: ", response)

6
Answer:  import numpy as np

reset_reward()
set_l2_distance_reward("dish rack", "dish plate")
set_obj_orientation_reward("dish plate", 0, 0)
set_obj_position_reward("dish plate", 0, -0.105, 0.83)

execute_plan(2)
